This notebook experiments with building out a fully customized TerriaJS catalog from a search result of ScienceBase Items, transforming sbJSON into the json catalog format used by TerriaJS. Resulting catalog JSON files are output to the local directory, deployed to GitHub, and then callable into an out of the box TerriaJS mapper.

In [1]:
%%bash

pip install sciencebasepy

In [0]:
import time
import os
import sciencebasepy
from getpass import getpass
import bs4
import requests
import json
import base64
import subprocess
import datetime
import getpass
import random
import string

from IPython.display import display

In [8]:
# Get what we want to work with using a ScienceBase /items/ query
sbURL = "https://www.sciencebase.gov/catalog/items?tags=Base+Map+Service&format=json&fields=title,body,webLinks&loadInstances=true"
#https://www.sciencebase.gov/catalog/item/4f552e93e4b018de15819c51"
#sbURL="https://www.sciencebase.gov/catalog/items?max=200&parentId=5644f3c1e4b0aafbcd0188f1&format=json&fields=title,body,distributionLinks,facets&&loadInstances=true"
sbJSON = requests.get(sbURL).json()
display (sbJSON)

{'items': [{'body': "USGS Topo is a topographic tile cache base map that combines the most current data (Boundaries, Names, Transportation, Elevation, Hydrography, Land Cover, and other themes) that make up The National Map. Contours generated for the US Topo product are included and are visible along with other data to the 1:18,000 zoom scale. This product is designed to provide a seamless view of the data, closely resembling The National Map's US Topo (GeoPDF) product at large scales, in a GIS accessible format.",
   'id': '5825a088e4b01fad86db6316',
   'link': {'rel': 'self',
    'url': 'https://www.sciencebase.gov/catalog/item/5825a088e4b01fad86db6316'},
   'relatedItems': {'link': {'rel': 'related',
     'url': 'https://www.sciencebase.gov/catalog/itemLinks?itemId=5825a088e4b01fad86db6316'}},
   'title': 'USGS Topo Base Map Service from The National Map',
   'webLinks': [{'hidden': False,
     'rel': 'related',
     'type': 'Online Link',
     'uri': 'http://viewer.nationalmap.gov

In [10]:
theCatalog = {}
theCatalog["corsDomains"] = ["sciencebase.gov"]
#theCatalog["baseMapName"] = ""
theCatalog["initialViewerMode"] = "2d"
theCatalog["services"] = []

theCatalog["homeCamera"] = {}
theCatalog["homeCamera"]["west"] = -175
theCatalog["homeCamera"]["east"] = -66
theCatalog["homeCamera"]["north"] = 72
theCatalog["homeCamera"]["south"] = 18

theCatalog["catalog"] = []

padusGroup = {}
padusGroup["name"] = "Wetland Observatory Data"
padusGroup["type"] = "group"
padusGroup["isOpen"] = "true"

padusGroup["items"] = []

for idx, item in enumerate(sbJSON["items"]):
    if 'body' not in item:
        continue
    if 'webLinks' not in item:
        continue
    thisItem = {}
    thisItem["id"] = item["id"]
    #thisItem["name"] = item["title"]
    thisItem["description"] = item["body"]
    serviceCount = 0

    for link in item["webLinks"]:
        if 'type' not in link:
          continue
        if link["type"] == "serviceCapabilitiesUrl":
            serviceCount = serviceCount + 1
            if serviceCount > 1:
                break
            thisItem["name"] = link["title"]
            thisItem["type"] = "wms-getCapabilities"
            fullUri=link["uri"]
            thisItem["url"] = link["uri"]
    if serviceCount == 0:
        continue
    padusGroup["items"].append(thisItem.copy())

theCatalog["catalog"].append(padusGroup.copy())

display (theCatalog)

f = open('catalog.json', 'wt', encoding='utf-8')
#f.write(json.dumps(theCatalog).replace(': "true"', ': true'))

{'catalog': [{'isOpen': 'true',
   'items': [{'description': "USGS Topo is a topographic tile cache base map that combines the most current data (Boundaries, Names, Transportation, Elevation, Hydrography, Land Cover, and other themes) that make up The National Map. Contours generated for the US Topo product are included and are visible along with other data to the 1:18,000 zoom scale. This product is designed to provide a seamless view of the data, closely resembling The National Map's US Topo (GeoPDF) product at large scales, in a GIS accessible format.",
     'id': '5825a088e4b01fad86db6316',
     'name': 'Topo Base Map Service (WMS)',
     'type': 'wms-getCapabilities',
     'url': 'http://basemap.nationalmap.gov/arcgis/services/USGSTopo/MapServer/WMSServer?request=GetCapabilities&service=WMS'},
    {'description': 'USGS Imagery Only is a tile cache base map of orthoimagery in The National Map visible to the 1:18,000 scale. Orthoimagery data are typically high resolution images that